In [92]:
# Comando para indexar os documentos
#  python3 -m pyserini.index.lucene   --collection JsonCollection     --generator DefaultLuceneDocumentGenerator      --threads 2         --input /home/tainan-santos/Documentos/PCC_ANALISE/data_json    --index /home/tainan-santos/Documentos/PCC_ANALISE/data_index   --storeDocvectors       --stemmer None      --language pt


In [93]:
# root = "/home/tainan/Documentos/PCC_ANALISES/data/data/"
# dataset_corpus = "base_20230428_douglas.csv"
# dataset_query = "query_expand_crf_novo_gerado_direto_do_microservoce_base_douglas_versao_final.csv"


In [94]:
# corpus = pd.read_csv(root + dataset_corpus, delimiter=",", encoding="utf-8")
# corpus.head(3)

In [95]:
import pandas as pd

querys = pd.read_csv("./data/dados-conle-anonimizado-assunto-notnull.csv", delimiter=";", encoding="utf-8")
querys

,NÚMERO-PROPOSIÇÃOSILEG,TxtAssunto
0,PEC 151/2019,Solicito a consultoria elaboração de Projeto d...
1,PL 1036/2019,"tese Conamat (arts. 545, 582 e 602 da CLT ..."
2,PL 1037/2019,art. 394-A da CLT - insalubridade trabalhadora...
3,PL 1075/2019,"Solicito a analise do Projeto de Lei, anexo, p..."
4,PL 1076/2019,Elaboração de Lei Federal que obrigue todos os...
...,...,...
290,PL 914/2019,Preparar Projeto de Lei para tratar da destina...
291,PL 927/2019,Obriga as instituições e comércios a expor os...
292,PL 947/2019,SOLICITO A FEITURA DE UM PROJETO DE LEI QUE PE...
293,PL 993/2019,"Solicito reelaboração do PL 10235/2018, para r..."


## RM3

In [96]:
from pyserini.search import LuceneSearcher

index_path = './data_index'
searcher = LuceneSearcher(index_path)
searcher.set_rm3(10, 10, 0.5)


def rm3_query_expansion(query):
    hits = searcher.search(query)
    num_results = min(10, len(hits))

    if num_results > 0:
        feedback_term = searcher.get_feedback_terms(query)

        sorted_terms = sorted(feedback_term.items(), key=lambda x: x[1], reverse=True)
        result = [term for term, relevance in sorted_terms]
        expanded_query = query + ' ' + ' '.join(result)
        return expanded_query

    return query

In [97]:
querys['query_exp'] = querys['TxtAssunto'].apply(rm3_query_expansion)


In [98]:
querys = querys[['NÚMERO-PROPOSIÇÃOSILEG', 'TxtAssunto', 'query_exp']]
querys = querys.rename(columns={"NÚMERO-PROPOSIÇÃOSILEG": "numero", "TxtAssunto": "text", "query_exp": "query_exp"})

In [99]:
querys.to_csv('resultados_rm3/rm3_new/result_rm3_analise_so_com_feedback_terms01.csv', index=False)